Load all necessary packages

In [ ]:
!pip3 install pydub

import os
import torch
import torchaudio
import requests
import pickle
from scipy import signal
from IPython.display import display, Audio, clear_output

# Install audio tools for colab
if not os.path.exists('silero-models'):
  !git clone -q --depth 1 https://github.com/snakers4/silero-models

%cd silero-models

from colab_utils import (record_audio,
                         audio_bytes_to_np,
                         upload_audio)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/silero-models/silero-models


Record audio

In [ ]:
record_seconds  = 4
sample_rate     = 16000

clear_output()
audio = record_audio(record_seconds).numpy()

# resample and convert to FloatTensor
audio = torch.FloatTensor(signal.resample(audio, sample_rate*record_seconds))

# apply VAD
audio = torchaudio.functional.vad(torch.flip(audio, ([0])), sample_rate=8000)
audio = torchaudio.functional.vad(torch.flip(audio, ([0])), sample_rate=8000)

# show recorded data
print('Check that the clip sounds okay')
display(Audio(data=audio, rate = sample_rate, autoplay=True))

Starting recording for 4 seconds...


<IPython.core.display.Javascript object>

Finished recording!
Check that the clip sounds okay


Send to server and wait for output

In [ ]:
header = {"Content-type": "application/x-www-form-urlencoded",
          "Accept": "text/plain"} 

# POST to server and wait for output
r = requests.post("http://SERVER:PORT/query-window", data=pickle.dumps(audio), headers=header)

# Print the returned text
print('Predicted text is',r.text)

Predicted text is 오후 4시에 예약 잡아주세요
